In [2]:
import keras
from keras.utils import np_utils
import numpy as np
from keras.layers import Activation,Dense,Flatten,Conv2D,MaxPooling2D,Dropout,BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler

Using TensorFlow backend.


In [3]:
def lr_schedule(epoch):
    lrate=0.0001
    if epoch > 75:
        lrate=0.0005
    if epoch > 100:
        lrate=0.0003
    return lrate

In [3]:
(xtrain,ytrain),(xtest,ytest)=cifar10.load_data()
xtrain=xtrain.astype('float32')
xtest=xtest.astype('float32')

In [4]:
xtrain.shape

(50000, 32, 32, 3)

In [7]:
#z-score - normalization
mean=np.mean(xtrain,axis=(0,1,2,3))
std=np.std(xtrain,axis=(0,1,2,3))
xtrain=(xtrain-mean)/(std+1e-7)
xtest=(xtest-mean)/(std+1e-7)

In [11]:
num_classes=10
ytrain=np_utils.to_categorical(ytrain,num_classes)
ytest=np_utils.to_categorical(ytest,num_classes)

In [20]:
weight_decay=1e-4

In [21]:
model=Sequential()

In [25]:
model.add(Conv2D(32,(3,3), padding='same',kernel_regularizer=regularizers.l2(weight_decay),input_shape=xtrain.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128,(3,3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes,activation='softmax'))

model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_

In [27]:
datagen=ImageDataGenerator(rotation_range=45,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True)
datagen.fit(xtrain)

In [30]:
#training

batch_size=64
opt_rms=keras.optimizers.rmsprop(lr=0.0001,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(xtrain,ytrain, batch_size=batch_size),steps_per_epoch=xtrain.shape[0] // batch_size,epochs=30,verbose=1,validation_data=(xtest,ytest),callbacks=[LearningRateScheduler(lr_schedule)])

model_json=model.to_json()
with ('model.json', 'w') as f:
    f.write(model_json)
model.save_weights('model.h5')

#testing
scores=model.evaluate(xtest, ytest,batch_size=128,verbose=1)
print('\n Test Result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
781/781 [==============================] - 562s 720ms/step - loss: 2.4173 - acc: 0.2942 - val_loss: 1.7277 - val_acc: 0.4408
Epoch 2/30
512/781 [==================>...........] - ETA: 3:01 - loss: 1.9673 - acc: 0.3764

KeyboardInterrupt: 

In [ ]:
from matplotlib.pyplot import plt

def show_images(X):
    plt.figure(1)
    k=0
    for i in range(0,4):
        for j in range(0,4):
            plt.subplot2grid((4,4),(i,j))
            plt.imshow((X[k]))
            k+=1
    plt.show()
            

In [ ]:
(xtrain,ytrain),(xtest,ytest)=cifar10.load_data()
xtrain=xtrain.astype('float32')
xtest=xtest.astype('float32')

#z-score - normalization
mean=np.mean(xtrain,axis=(0,1,2,3))
std=np.std(xtrain,axis=(0,1,2,3))
xtrain=(xtrain-mean)/(std+1e-7)
xtest=(xtest-mean)/(std+1e-7)

show_images(xtest[:16])

json_file=open(model.json,'r')
loaded_model=json_file.read()
json_file.close()

from keras.models import model_from_json
model=model_from_json(loaded_model)
model.load_weights('model.h5')

labels=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

indices=np.argmax(model.predict(xtest[:16]),1)

print([labels[x] for x in indices])